In [1]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas
from tqdm import tqdm

In [2]:
dataset_name = "presencesw/vinli_3_label"
label = 'contradiction'
dataset = load_dataset(dataset_name)

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 18282
    })
    test: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2264
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2255
    })
})


In [4]:
def convert_entailment(exmaples):
    labels = exmaples['gold_label']
    labels = [f"not_{label}" if i != label else label for i in labels]
    exmaples['gold_label'] = labels
    return exmaples

In [5]:
# this step convert dataset to 2 labels
converted_dataset = dataset.map(convert_entailment, batched=True)

# check number samples in dataset train
converted_dataset['train']

Dataset({
    features: ['gold_label', 'sentence1', 'sentence2'],
    num_rows: 18282
})

In [6]:
# Divine the dataset to 2 sets `entailment` and `not entailment`
new_dataset_entail = converted_dataset.filter(lambda example: example['gold_label'] == label)['train']
new_dataset_not_entail = converted_dataset.filter(lambda example: example['gold_label'] == f'not_{label}')['train']

Filter:   0%|          | 0/18282 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2255 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18282 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [7]:
# convert dataset to pandas for next step
new_dataset_entail = Dataset.to_pandas(new_dataset_entail)
# duplicate dataset not entailment
new_dataset_not_entail1 = Dataset.to_pandas(new_dataset_not_entail)
new_dataset_not_entail2 = Dataset.to_pandas(new_dataset_not_entail)

In [8]:
# merge 2 dataset not entailment for the target
j = 0
for i in tqdm(range(len(new_dataset_not_entail1))):
    temp = set()
    # check = False
    # for j in range(len(new_dataset_not_entail2)):
    for index in range(100):
        new_j = j + index
        if new_j == len(new_dataset_not_entail1):
            j = new_j
            break
        if new_dataset_not_entail1.iloc[i]['sentence1'] != new_dataset_not_entail2.iloc[new_j]['sentence1']:
            j = new_j
            break
        temp.add(new_dataset_not_entail2.iloc[new_j]['sentence2'])
    # new_dataset_not_entail1.iloc[i]['sentence2'] = temp
    # temp = [val for val in temp]
    new_dataset_not_entail1.at[i, 'sentence2'] = temp

100%|██████████| 12188/12188 [00:01<00:00, 6472.70it/s]


In [9]:
# dataset_not_entail = new_dataset_not_entail1.drop_duplicates(subset=['sentence2'])
# print(len(dataset_not_entail))
dataset_not_entail = new_dataset_not_entail1[new_dataset_not_entail1['sentence2'] != set()]
print(len(dataset_not_entail))

3044


In [10]:
dataset_not_entail = dataset_not_entail.reset_index(drop=True)
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_contradiction,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",{Tập đoàn Trung Nam có ý định bán toàn bộ cổ p...
1,not_contradiction,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",{Nhiều người mong muốn được giữ quyền điều hàn...
2,not_contradiction,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",{Số cổ phần của tập đoàn Trung Nam tại Nhà máy...
3,not_contradiction,"Dự án Nhà máy Điện gió Trung Nam, tổng số vốn ...",{Vốn đầu tư của Nhà máy Điện gió Trung Nam sẽ ...
4,not_contradiction,"Từ nay tới năm 2028, dòng vốn đầu tư được dự b...",{Những hoạt động đi lại còn hạn chế là do dịch...
...,...,...,...
3039,not_contradiction,Ổ SSD dùng để khai thác tiền số Chia chỉ còn t...,{Việc khai thác tiền số Chia đã làm giảm tuổi ...
3040,not_contradiction,"Khi khai thác Chia, những ổ cứng dung lượng lớ...",{Một ổ cứng sử dụng với mục đích thông thường ...
3041,not_contradiction,Ổ cứng thường được hãng bảo hành từ ba đến năm...,{Người mua ổ cứng sẽ được bảo hành miễn phí từ...
3042,not_contradiction,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...","{Ổ SSD có tốc độ xử lý cao hơn ổ HDD., Nhiều n..."


In [11]:
temp = []
for index1 in tqdm(range(len(new_dataset_entail))):
    # print(new_dataset_entail.iloc[index1]['sentence1'])
    for index2 in range(len(dataset_not_entail)):
        if new_dataset_entail.iloc[index1]['sentence1'] == dataset_not_entail.iloc[index2]['sentence1']:
            temp.append(dataset_not_entail.iloc[index2]['sentence2'])
            break

100%|██████████| 6094/6094 [04:56<00:00, 20.56it/s]


In [12]:
print(len(temp))

6094


In [13]:
new_dataset_entail['negative'] = temp
# new_dataset_entail

In [14]:
new_dataset = Dataset.from_pandas(new_dataset_entail)
new_dataset = new_dataset.rename_column("sentence1", "anchor")
new_dataset = new_dataset.rename_column("sentence2", "positive")
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 6094
})

In [15]:
def convert_dataset(example):
    # print(example['negative'])
    number = len(example['negative'][0])
    # print(gold_label)
    gold_label = example['gold_label'] * number
    anchor = example['anchor'] * number
    positive = example['positive'] * number
    return {'gold_label': gold_label, 'anchor': anchor, 
            'positive': positive, 'negative': example['negative'][0]}

        
new_dataset = new_dataset.map(
    lambda example: convert_dataset(example),                   
    remove_columns=['gold_label', 'anchor', 'positive', 'negative'],
    batched=True,
    batch_size=1
)

Map:   0%|          | 0/6094 [00:00<?, ? examples/s]

In [16]:
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 24406
})

In [17]:
print(['']*5)

['', '', '', '', '']


In [18]:
def add_columns(examples):
    # print(len(examples))
    length = len(examples['gold_label'])
    examples['negative'] = [''] * length
    return examples

In [19]:
converted_dataset = converted_dataset.rename_column("sentence1", "anchor")
converted_dataset = converted_dataset.rename_column("sentence2", "positive")

In [20]:
converted_dataset = converted_dataset.map(add_columns, batched=True)

Map:   0%|          | 0/18282 [00:00<?, ? examples/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

Map:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [21]:
all_data = DatasetDict()
all_data['train'] = new_dataset
all_data['validation'] = converted_dataset['dev']
all_data['test'] = converted_dataset['test']

In [22]:
all_data.push_to_hub(f"{dataset_name}_{label}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/392 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/presencesw/vinli_3_label_contradiction/commit/fc48d832b9910ad0e5f7d8464c60781e97b6204b', commit_message='Upload dataset', commit_description='', oid='fc48d832b9910ad0e5f7d8464c60781e97b6204b', pr_url=None, pr_revision=None, pr_num=None)